In [1]:
import os
import sys 
from flask import Flask
from flask import request
from flask_api import status
import psycopg2
import json
import copy
from gensim import corpora, models, similarities
import jieba
import stanza
from spacy_stanza import StanzaLanguage

snlp = stanza.Pipeline(lang="es")
nlp = StanzaLanguage(snlp)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

2020-10-02 11:29:50 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2020-10-02 11:29:50 INFO: Use device: cpu
2020-10-02 11:29:50 INFO: Loading: tokenize
2020-10-02 11:29:50 INFO: Loading: mwt
2020-10-02 11:29:50 INFO: Loading: pos
2020-10-02 11:29:51 INFO: Loading: lemma
2020-10-02 11:29:51 INFO: Loading: depparse
2020-10-02 11:29:52 INFO: Loading: ner
2020-10-02 11:29:53 INFO: Done loading processors!


In [2]:
def getIntension(texts,keyword,intension):
    texts = [jieba.lcut(text) for text in texts]
    dictionary = corpora.Dictionary(texts)
    feature_cnt = len(dictionary.token2id)
    corpus = [dictionary.doc2bow(text) for text in texts]
    tfidf = models.TfidfModel(corpus) 
    kw_vector = dictionary.doc2bow(jieba.lcut(keyword))
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
    sim = index[tfidf[kw_vector]]
    for i in range(len(sim)):
        if sim[i] >= .8:

            return intension
            break
    return 'false'  

In [3]:
def abrirJson(name,path=os.getcwd()+"/" ):
    with open(path+name,encoding='utf-8') as data_file:  
        diccionario = json.load(data_file)
    return diccionario

In [5]:
app = Flask(__name__)
@app.route('/nlu/<oracion>',methods=['POST'])
def NLU(oracion):
    print("NLU")
    print("oracion:",oracion)
    informacion = request.get_json()["informacion"]   
    try:
        respuesta=dict()
        i=0
        intension = 'false'
        while i<len(informacion) and intension == 'false':           
            intension = getIntension(informacion[i]["examples"],oracion,(informacion[i]["intension"]))
            i=i+1
            if intension == 'false':
                pass
            else:
                break
        doc = nlp(oracion)
        entidades=dict()
        for token in doc:
            if token.ent_type_:
                entidades.update({token.text:token.ent_type_})       
        respuesta.update({"entidades":entidades})
        respuesta.update({"intension":intension})
        return respuesta        
    except (Exception, psycopg2.Error) as error :
        print("error",error)
        return oracion,status.HTTP_404_NOT_FOUND

In [6]:
if __name__ == "__main__":    
    app.run(host='localhost',port='8282')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8282/ (Press CTRL+C to quit)
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


NLU
oracion: ¿cuántos años tienes


Loading model cost 0.676 seconds.
Prefix dict has been built successfully.
127.0.0.1 - - [02/Oct/2020 11:30:30] "POST /nlu/¿cuántos%20años%20tienes HTTP/1.1" 200 -


NLU
oracion: ¿cuántos años tienes


...

127.0.0.1 - - [02/Oct/2020 12:59:06] "POST /nlu/¿quién%20eres HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2020 13:01:37] "POST /nlu/hola HTTP/1.1" 200 -


NLU
oracion: hola
NLU
oracion: ¿cuántos años tienes


127.0.0.1 - - [02/Oct/2020 13:02:17] "POST /nlu/¿cuántos%20años%20tienes HTTP/1.1" 200 -


NLU
oracion: ¿qué edad tienes


127.0.0.1 - - [02/Oct/2020 13:11:38] "POST /nlu/¿qué%20edad%20tienes HTTP/1.1" 200 -
